In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
# Define the filename
filename = 'benchmark_results/results_gop4_inter.npy'
data = np.load(filename, allow_pickle=True).item()
# print(data)
videos = ['Beauty', 'Bosphorus', 'ShakeNDry', 'HoneyBee','Jockey', 'ReadySteadyGo', 'YachtRide']
video_df= []
for video in videos:
    video_df.append(pd.DataFrame(data[video]) )
    
metrics = pd.concat(video_df).groupby(level=0)[['bpp_inter', 'LPIPS', 'MS-SSIM','PSNR','FID']].mean()
ours = metrics.iloc[4:]
hevc = metrics.iloc[0:4]

In [5]:
ours

,bpp_inter,LPIPS,MS-SSIM,PSNR,FID
4,0.007142,0.106987,0.879951,23.826525,1.329397
5,0.006269,0.106955,0.879959,23.827501,1.326203
6,0.005952,0.107067,0.879882,23.826249,1.323439


In [7]:
# 1. Prepare data and create the output folder
df = hevc.sort_values('bpp_total')
highlight_point = ours
output_folder = 'benchmark_results/gop4_plots'
os.makedirs(output_folder, exist_ok=True) # Creates the folder if it doesn't exist

# 2. Define the columns for plotting
x_col = 'bpp_total'
y_cols = ['LPIPS', 'PSNR', 'MS-SSIM', 'FID']

# 3. Loop through each metric to create and save a separate plot
for y_col in y_cols:
    # Create a new, single figure for each iteration
    fig, ax = plt.subplots(figsize=(8, 6))

    # Plot the original data points on the new axis
    ax.plot(df[x_col], df[y_col], label='HEVC Inter Bpp')

    # Plot your specific point on top
    ax.plot(highlight_point[x_col], highlight_point[y_col],
               color='red',
               # marker='*',
               # s=250,
               # zorder=5,
               label='BiFCA Inter Bpp')

    # Set titles, labels, and legend
    ax.set_title(f'{y_col.upper()} vs {x_col.upper()}', fontsize=16)
    ax.set_xlabel(x_col.upper(), fontsize=12)
    ax.set_ylabel(y_col.upper(), fontsize=12)
    ax.grid(True, linestyle='--', alpha=0.6)
    ax.legend()

    # Define the full path and save the figure
    filename = f"{y_col}_vs_{x_col}.svg"
    full_path = os.path.join(output_folder, filename)
    plt.savefig(full_path, format='svg', dpi=800)

    # Close the figure to free up memory before the next loop
    plt.close(fig)

print(f"Successfully saved 4 plots in the '{output_folder}' directory.")

Successfully saved 4 plots in the 'benchmark_results/gop4_plots' directory.


In [20]:
import os
import pandas as pd
from pathlib import Path

def read_compression_report(file_path):
    try:
        with open(file_path, 'r') as file:
            data = {'Intra bytes': 0, 'Inter bytes': 0, 'Total bytes': 0}
            for line in file:
                line = line.strip()
                if line.startswith('Intra bytes:'):
                    data['Intra bytes'] = int(line.split(': ')[1])
                elif line.startswith('Inter bytes:'):
                    data['Inter bytes'] = int(line.split(': ')[1])
                elif line.startswith('Total bytes:'):
                    data['Total bytes'] = int(line.split(': ')[1])
        return data
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None


base_path = Path('hevc_class_b_gop4')
videos =  [d for d in base_path.iterdir() if d.is_dir()]
rows =[]
for video in videos:
    bpps = os.listdir(video)
    for bpp in bpps:
        file_path = video / bpp / 'intra_inter_storage.txt'
        print(file_path)
        data = read_compression_report(file_path)
        data["Video"] = video.stem
        data['bpp'] = bpp.split('bpp_')[1]
        rows.append(data)

benchmark_results/hevc_gop4_new/Beauty/bpp_0.1/intra_inter_storage.txt
benchmark_results/hevc_gop4_new/Beauty/bpp_0.05/intra_inter_storage.txt
benchmark_results/hevc_gop4_new/Beauty/bpp_0.006/intra_inter_storage.txt
benchmark_results/hevc_gop4_new/Beauty/bpp_0.01/intra_inter_storage.txt
benchmark_results/hevc_gop4_new/Bosphorus/bpp_0.1/intra_inter_storage.txt
benchmark_results/hevc_gop4_new/Bosphorus/bpp_0.05/intra_inter_storage.txt
benchmark_results/hevc_gop4_new/Bosphorus/bpp_0.006/intra_inter_storage.txt
benchmark_results/hevc_gop4_new/Bosphorus/bpp_0.01/intra_inter_storage.txt
benchmark_results/hevc_gop4_new/HoneyBee/bpp_0.1/intra_inter_storage.txt
benchmark_results/hevc_gop4_new/HoneyBee/bpp_0.05/intra_inter_storage.txt
benchmark_results/hevc_gop4_new/HoneyBee/bpp_0.006/intra_inter_storage.txt
benchmark_results/hevc_gop4_new/HoneyBee/bpp_0.01/intra_inter_storage.txt
benchmark_results/hevc_gop4_new/YachtRide/bpp_0.1/intra_inter_storage.txt
benchmark_results/hevc_gop4_new/YachtRide/

In [21]:
df = pd.DataFrame(rows)

In [22]:
# Initialize the dictionary
hevc_bits_gop4 = {'inter': {}, 'intra': {}}
videos = df['Video'].unique()
bpp_dict = [0.006, 0.01, 0.05, 0.1]

# Populate the dictionary
for video in videos:
    hevc_bits_gop4['inter'][video] = []
    hevc_bits_gop4['intra'][video] = []
    for bpp in bpp_dict:
        row = df[(df['Video'] == video) & (df['bpp'] == str(bpp))]
        if not row.empty:
            hevc_bits_gop4['inter'][video].append(row['Inter bytes'].iloc[0])
            hevc_bits_gop4['intra'][video].append(row['Intra bytes'].iloc[0])
        else:
            hevc_bits_gop4['inter'][video].append(0)
            hevc_bits_gop4['intra'][video].append(0)

# Print updated dictionary
print(hevc_bits_gop4)

{'inter': {'Beauty': [14274, 25046, 202156, 359468], 'Bosphorus': [6302, 7627, 20080, 46135], 'HoneyBee': [5835, 5921, 11910, 20815], 'YachtRide': [10983, 15318, 135414, 408250], 'ReadySteadyGo': [19096, 15802, 58908, 137989], 'ShakeNDry': [8853, 10407, 77770, 381155], 'Jockey': [23276, 31554, 210572, 458851]}, 'intra': {'Beauty': [111217, 154065, 638052, 1530754], 'Bosphorus': [161389, 221594, 852442, 1662727], 'HoneyBee': [229275, 275556, 862427, 1643954], 'YachtRide': [169022, 227813, 826756, 1492341], 'ReadySteadyGo': [284820, 297002, 847547, 1526334], 'ShakeNDry': [225178, 277665, 877355, 1485597], 'Jockey': [172028, 229147, 770061, 1244686]}}
